#SmartOS – Voice/Chat-Based AI OS Assistant (Standalone Prototype)
**Author:** Gauransh Gautam

This prototype:
- Takes natural language commands (text).
- Detects intent (rule-based NLU).
- Executes safe OS/file actions (create/write files, open apps, connect/disconnect Wi-Fi [mock]).
- Logs results into JSON and Excel for evaluation.

In [8]:
from pathlib import Path
import subprocess, platform, time, json, os, re
import pandas as pd

# Setup folders
BASE_DIR = Path.cwd()
FILES_ROOT = BASE_DIR / "smartos_sandbox"
LOG_DIR = BASE_DIR / "smartos_logs"
FILES_ROOT.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_JSONL = LOG_DIR / "results.jsonl"
RESULTS_XLSX = LOG_DIR / "results.xlsx"

# Simple OS app command mapping
APP_COMMANDS = {
    "notepad": {"win": ["notepad.exe"], "mac": ["open", "-a", "TextEdit"], "linux": ["gedit"]},
    "calculator": {"win": ["calc.exe"], "mac": ["open", "-a", "Calculator"], "linux": ["gnome-calculator"]},
    "chrome": {"win": ["chrome.exe"], "mac": ["open", "-a", "Google Chrome"], "linux": ["google-chrome"]},
}

def _os_key():
    s = platform.system().lower()
    if "windows" in s: return "win"
    if "darwin" in s: return "mac"
    return "linux"

print("Sandbox folder:", FILES_ROOT)
print("Logs folder:", LOG_DIR)


Sandbox folder: C:\Users\Hp\smartos_sandbox
Logs folder: C:\Users\Hp\smartos_logs


In [9]:
INTENT_PATTERNS = [
    {"intent": "open_app", "pattern": r"\b(open|launch|start)\s+(?P<app>[a-zA-Z0-9_.-]+)\b"},
    {"intent": "create_file", "pattern": r"\b(create|make)\s+(file|document)\s+called\s+(?P<name>[\w.-]+)\b"},
    {"intent": "write_file", "pattern": r"\b(write|append)\s+(?P<text>.+)\s+to\s+(?P<name>[\w.-]+)\b"},
    {"intent": "connect_wifi", "pattern": r"\b(connect)\s+(to\s+)?wi[- ]?fi\b"},
    {"intent": "disconnect_wifi", "pattern": r"\b(disconnect)\s+(from\s+)?wi[- ]?fi\b"},
    {"intent": "help", "pattern": r"\b(help|what can you do)\b"},
]

def parse_intent(utterance: str):
    u = utterance.strip().lower()
    for entry in INTENT_PATTERNS:
        m = re.search(entry["pattern"], u)
        if m:
            return {"intent": entry["intent"], "slots": m.groupdict(), "raw": utterance}
    return {"intent": "clarify", "slots": {}, "raw": utterance}


In [10]:
def open_app(app_name):
    key = _os_key()
    norm = app_name.lower()
    if norm in APP_COMMANDS and key in APP_COMMANDS[norm]:
        try:
            subprocess.Popen(APP_COMMANDS[norm][key])
            return {"ok": True, "message": f"Launched {app_name}"}
        except Exception as e:
            return {"ok": False, "message": f"Error launching {app_name}: {e}"}
    return {"ok": False, "message": f"Unknown app {app_name}"}

def create_file(name):
    p = (FILES_ROOT / name).resolve()
    try:
        p.touch(exist_ok=True)
        return {"ok": True, "message": f"Created file {p}"}
    except Exception as e:
        return {"ok": False, "message": f"Error creating file: {e}"}

def write_file(name, text):
    p = (FILES_ROOT / name).resolve()
    try:
        with open(p, "a", encoding="utf-8") as f:
            f.write(text + "\n")
        return {"ok": True, "message": f"Wrote to {p}"}
    except Exception as e:
        return {"ok": False, "message": f"Error writing file: {e}"}

def connect_wifi():
    time.sleep(0.2)
    return {"ok": True, "message": "Wi-Fi connected (mock)"}

def disconnect_wifi():
    time.sleep(0.2)
    return {"ok": True, "message": "Wi-Fi disconnected (mock)"}

def execute(parsed):
    intent = parsed["intent"]
    slots = parsed.get("slots", {})
    if intent == "open_app": return open_app(slots.get("app",""))
    if intent == "create_file": return create_file(slots.get("name","file.txt"))
    if intent == "write_file": return write_file(slots.get("name","file.txt"), slots.get("text",""))
    if intent == "connect_wifi": return connect_wifi()
    if intent == "disconnect_wifi": return disconnect_wifi()
    if intent == "help": return {"ok": True, "message": "I can open apps, create/write files, and manage Wi-Fi (mock)."}
    return {"ok": True, "message": "Sorry, I didn’t understand. Try: 'open notepad' or 'create file called test.txt'."}


In [11]:
def run_and_log(test_name, description, result):
    entry = {
        "test_name": test_name,
        "description": description,
        "status": "Success" if result.get("ok") else "Fail",
        "error": None if result.get("ok") else result.get("message"),
        "latency_ms": int(time.time()*1000),
    }
    with open(RESULTS_JSONL, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")
    return entry

def export_to_excel():
    rows = []
    if RESULTS_JSONL.exists():
        with open(RESULTS_JSONL, "r") as f:
            for line in f:
                rows.append(json.loads(line))
    df = pd.DataFrame(rows)
    df.to_excel(RESULTS_XLSX, index=False)
    return RESULTS_XLSX, df


In [12]:
def run_command(utterance):
    parsed = parse_intent(utterance)
    result = execute(parsed)
    entry = run_and_log(parsed["intent"], utterance, result)
    print(f"User: {utterance}")
    print(f"SmartOS: {result['message']}")
    return entry

# Demo commands
demo = [
    "create file called notes.txt",
    "write patient has fever to notes.txt",
    "connect to wifi",
    "how are you?"
]

for cmd in demo:
    run_command(cmd)


User: create file called notes.txt
SmartOS: Created file C:\Users\Hp\smartos_sandbox\notes.txt
User: write patient has fever to notes.txt
SmartOS: Wrote to C:\Users\Hp\smartos_sandbox\notes.txt
User: connect to wifi
SmartOS: Wi-Fi connected (mock)
User: how are you?
SmartOS: Sorry, I didn’t understand. Try: 'open notepad' or 'create file called test.txt'.


In [13]:
xlsx_path, df = export_to_excel()
print("Logs saved at:", xlsx_path)
df


Logs saved at: C:\Users\Hp\smartos_logs\results.xlsx


,test_name,description,status,error,screenshot,started_at,ended_at,latency_ms
0,create_file,create file called notes.txt,Success,None,NaN,1.756379e+12,1.756379e+12,0
1,write_file,write patient has fever to notes.txt,Success,None,NaN,1.756379e+12,1.756379e+12,0
2,open_app,open notepad,Success,None,NaN,1.756379e+12,1.756379e+12,0
3,connect_wifi,connect to wifi,Success,None,NaN,1.756379e+12,1.756379e+12,0
4,chitchat,how are you?,Success,None,NaN,1.756379e+12,1.756379e+12,0
5,create_file,create file called notes.txt,Success,None,NaN,NaN,NaN,1756379099394
6,write_file,write patient has fever to notes.txt,Success,None,NaN,NaN,NaN,1756379099418
7,connect_wifi,connect to wifi,Success,None,NaN,NaN,NaN,1756379099621
8,clarify,how are you?,Success,None,NaN,NaN,NaN,1756379099623


In [14]:
# Cell 8 — show created files & logs (helpful for demo)
print("Sandbox files:")
for p in FILES_ROOT.glob("*"):
    print("-", p.name)
print("\nLog files:")
for p in LOG_DIR.glob("*"):
    print("-", p.name)

Sandbox files:
- notes.txt

Log files:
- results.jsonl
- results.xlsx


In [15]:
# Show what's inside notes.txt (or any file you created)
with open(FILES_ROOT / "notes.txt", "r", encoding="utf-8") as f:
    print(f.read())

patient has fever
patient has fever



In [16]:
# Display the Excel log in Jupyter
import pandas as pd

df = pd.read_excel(RESULTS_XLSX)
df

,test_name,description,status,error,screenshot,started_at,ended_at,latency_ms
0,create_file,create file called notes.txt,Success,NaN,NaN,1.756379e+12,1.756379e+12,0
1,write_file,write patient has fever to notes.txt,Success,NaN,NaN,1.756379e+12,1.756379e+12,0
2,open_app,open notepad,Success,NaN,NaN,1.756379e+12,1.756379e+12,0
3,connect_wifi,connect to wifi,Success,NaN,NaN,1.756379e+12,1.756379e+12,0
4,chitchat,how are you?,Success,NaN,NaN,1.756379e+12,1.756379e+12,0
5,create_file,create file called notes.txt,Success,NaN,NaN,NaN,NaN,1756379099394
6,write_file,write patient has fever to notes.txt,Success,NaN,NaN,NaN,NaN,1756379099418
7,connect_wifi,connect to wifi,Success,NaN,NaN,NaN,NaN,1756379099621
8,clarify,how are you?,Success,NaN,NaN,NaN,NaN,1756379099623


In [17]:
# List files inside sandbox and logs
print("Sandbox files:", list(FILES_ROOT.iterdir()))
print("Log files:", list(LOG_DIR.iterdir()))

Sandbox files: [WindowsPath('C:/Users/Hp/smartos_sandbox/notes.txt')]
Log files: [WindowsPath('C:/Users/Hp/smartos_logs/results.jsonl'), WindowsPath('C:/Users/Hp/smartos_logs/results.xlsx')]
